In [ ]:
!wget https://lodmedia.hb.bizmrg.com/case_files/868816/train_dataset_train.csv
!wget https://lodmedia.hb.bizmrg.com/case_files/868816/test_dataset_test.csv
!wget https://lodmedia.hb.bizmrg.com/cases/868816/%D0%AF%D0%BA%D1%83%D1%82%D1%81%D0%BA.zip
!unzip /content/Якутск.zip

In [3]:
import time
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import dbscan,DBSCAN
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, SGDRegressor, RidgeClassifier, LogisticRegressionCV,Ridge,QuantileRegressor,PassiveAggressiveClassifier,PoissonRegressor
from sklearn.ensemble import ExtraTreesRegressor,ExtraTreesClassifier,RandomForestClassifier,VotingClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,StackingRegressor,BaggingClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.svm import LinearSVC,LinearSVR,SVR,NuSVR
from sklearn.decomposition import TruncatedSVD,PCA,FactorAnalysis,IncrementalPCA,FastICA,KernelPCA,NMF
from sklearn.preprocessing import RobustScaler,QuantileTransformer,PowerTransformer,PolynomialFeatures,KBinsDiscretizer,StandardScaler,OneHotEncoder,OrdinalEncoder,FunctionTransformer,MaxAbsScaler,MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline,FeatureUnion,TransformerMixin
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,LocalOutlierFactor
from sklearn.model_selection import train_test_split,ShuffleSplit,StratifiedShuffleSplit,TimeSeriesSplit,GridSearchCV,KFold,StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn import set_config
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,roc_auc_score,accuracy_score,f1_score,classification_report,recall_score,make_scorer

In [ ]:
# загрузка данных
train=pd.read_csv('/content/train_dataset_train.csv')
test=pd.read_csv('/content/test_dataset_test.csv')
calls=pd.read_csv('/content/Calls.csv')
connectiontime=pd.read_csv('/content/ConnectionTime.csv')
education=pd.read_csv('/content/Education.csv')
skud=pd.read_csv('/content/SKUD.csv')
tasks=pd.read_csv('/content/Tasks.csv')
timennetwork=pd.read_csv('/content/TimenNetwork.csv')
workingday=pd.read_csv('/content/WorkingDay.csv')

In [5]:
# генерация переменных
train.index=train.id

train['CallTime']=calls['CallTime'].str.replace(',','.').astype('float').groupby(calls['id']).median()
train['NumberOfCalls']=calls['NumberOfCalls'].groupby(calls['id']).median()
train['Время опоздания']=connectiontime['Время опоздания'].str.replace(',','.').astype('float').fillna(0).groupby(connectiontime['id']).median()
train['Длительность общая']=skud['Длительность общая'].str.replace(',','.').astype('float').fillna(0).groupby(skud['id']).median()
train['Длительность раб.дня без обеда']=skud['Длительность раб.дня без обеда'].str.replace(',','.').astype('float').fillna(0).groupby(skud['id']).median()
train['Просрочено, дней']=tasks['Просрочено, дней'].fillna(0).groupby(tasks['id']).median()
train['monitor_Time']=timennetwork['monitor_Time'].fillna(0).groupby(timennetwork['id']).median()
train['activeTime']=workingday['activeTime'].groupby(workingday['id']).median()
train['monitorTime']=workingday['monitorTime'].groupby(workingday['id']).median()

train['Вид учета времени']=calls['Вид учета времени'].astype('str').groupby(calls['id']).aggregate(' '.join)
train['InOut']=calls['InOut'].groupby(calls['id']).aggregate(' '.join)
train['Табельный номер руководителя']=education['Табельный номер руководителя'].astype('str').groupby(education['id']).aggregate(' '.join)
train['Вид образования']=education['Вид образования'].astype('str').groupby(education['id']).aggregate(' '.join)
train['Специальность']=education['Специальность'].astype('str').groupby(education['id']).aggregate(' '.join)
train['Вид документа']=tasks['Вид документа'].astype('str').groupby(tasks['id']).aggregate(' '.join)
train['Состояние задания']=tasks['Состояние задания'].astype('str').groupby(tasks['id']).aggregate(' '.join)

train[['Вид учета времени','InOut','Вид образования','Специальность','Табельный номер руководителя','Вид документа','Состояние задания']]=train[[
    'Вид учета времени','InOut','Вид образования','Специальность','Табельный номер руководителя','Вид документа','Состояние задания']].astype('str')
train[['CallTime', 'NumberOfCalls', 'Время опоздания', 
       'Длительность общая', 'Длительность раб.дня без обеда', 
       'Просрочено, дней']]=train[['CallTime', 'NumberOfCalls', 'Время опоздания', 
       'Длительность общая', 'Длительность раб.дня без обеда', 
       'Просрочено, дней']].fillna(-1)

In [10]:
# генерация переменных
test.index=test.id

test['CallTime']=calls['CallTime'].str.replace(',','.').astype('float').groupby(calls['id']).median()
test['NumberOfCalls']=calls['NumberOfCalls'].groupby(calls['id']).median()
test['Время опоздания']=connectiontime['Время опоздания'].str.replace(',','.').astype('float').fillna(0).groupby(connectiontime['id']).median()
test['Длительность общая']=skud['Длительность общая'].str.replace(',','.').astype('float').fillna(0).groupby(skud['id']).median()
test['Длительность раб.дня без обеда']=skud['Длительность раб.дня без обеда'].str.replace(',','.').astype('float').fillna(0).groupby(skud['id']).median()
test['Просрочено, дней']=tasks['Просрочено, дней'].fillna(0).groupby(tasks['id']).median()
test['monitor_Time']=timennetwork['monitor_Time'].fillna(0).groupby(timennetwork['id']).median()
test['activeTime']=workingday['activeTime'].groupby(workingday['id']).median()
test['monitorTime']=workingday['monitorTime'].groupby(workingday['id']).median()

test['Вид учета времени']=calls['Вид учета времени'].astype('str').groupby(calls['id']).aggregate(' '.join)
test['InOut']=calls['InOut'].groupby(calls['id']).aggregate(' '.join)
test['Табельный номер руководителя']=education['Табельный номер руководителя'].astype('str').groupby(education['id']).aggregate(' '.join)
test['Вид образования']=education['Вид образования'].astype('str').groupby(education['id']).aggregate(' '.join)
test['Специальность']=education['Специальность'].astype('str').groupby(education['id']).aggregate(' '.join)
test['Вид документа']=tasks['Вид документа'].astype('str').groupby(tasks['id']).aggregate(' '.join)
test['Состояние задания']=tasks['Состояние задания'].astype('str').groupby(tasks['id']).aggregate(' '.join)

test[['Вид учета времени','InOut','Вид образования','Специальность','Табельный номер руководителя','Вид документа','Состояние задания']]=test[[
    'Вид учета времени','InOut','Вид образования','Специальность','Табельный номер руководителя','Вид документа','Состояние задания']].astype('str')
test[['CallTime', 'NumberOfCalls', 'Время опоздания', 
       'Длительность общая', 'Длительность раб.дня без обеда', 
       'Просрочено, дней']]=test[['CallTime', 'NumberOfCalls', 'Время опоздания', 
       'Длительность общая', 'Длительность раб.дня без обеда', 
       'Просрочено, дней']].fillna(-1)

In [8]:
# модель
pipe=Pipeline([('input',ColumnTransformer([
               ('text3',CountVectorizer(),'Табельный номер руководителя'),
               ('text6',CountVectorizer(),'Вид документа'),
               ('disc',KBinsDiscretizer(n_bins=3, strategy='kmeans'),['CallTime','Длительность общая'])])),
               ('tfidf',TfidfTransformer()),
               ('fs',SelectFromModel(LinearSVC(C=1,class_weight='balanced'),threshold='0.1*mean')),
               ('tfidf2',TfidfTransformer()),
               ('classifier',LinearSVC(C=1,class_weight='balanced'))])

In [11]:
# сохранение решения
test['type']=pipe.fit(train,train['type']).predict(test)
submit=test[['id','type']]
submit.to_csv('submit.csv',index=False)